Output: airport_cities.csv, flight_times_extended.csv
To create: dataset with cities, that are not matched between airport_cities and train_cities

# Comparing train and plane travel times

Import of libraries/flight data

In [3]:
import pandas as pd

flights = pd.read_csv('../datasets/flight_times.csv')
flights

,Unnamed: 0,route,time
0,0,"{'AAL', 'AMS'}",85.0
1,1,"{'AAQ', 'LED'}",170.0
2,2,"{'AMS', 'ABZ'}",85.0
3,3,"{'ABZ', 'LGW'}",95.0
4,4,"{'ACE', 'AGP'}",130.0
...,...,...,...
1464,1464,"{'ZRH', 'OTP'}",150.0
1465,1465,"{'ZRH', 'PRN'}",127.5
1466,1466,"{'ZRH', 'RVN'}",205.0
1467,1467,"{'ZRH', 'STR'}",45.0


In [51]:
flights.drop(flights.columns[0], axis=1, inplace=True)
flights

,route,time
0,"{'AAL', 'AMS'}",85.0
1,"{'AAQ', 'LED'}",170.0
2,"{'AMS', 'ABZ'}",85.0
3,"{'ABZ', 'LGW'}",95.0
4,"{'ACE', 'AGP'}",130.0
...,...,...
1464,"{'ZRH', 'OTP'}",150.0
1465,"{'ZRH', 'PRN'}",127.5
1466,"{'ZRH', 'RVN'}",205.0
1467,"{'ZRH', 'STR'}",45.0


In [52]:
flights['departure airport'] = flights['route'].apply(lambda x: x.split(',')[0].strip("{'"))
flights['arrival airport'] = flights['route'].apply(lambda x: x.split(',')[1].strip(" }'"))
flights

,route,time,departure airport,arrival airport
0,"{'AAL', 'AMS'}",85.0,AAL,AMS
1,"{'AAQ', 'LED'}",170.0,AAQ,LED
2,"{'AMS', 'ABZ'}",85.0,AMS,ABZ
3,"{'ABZ', 'LGW'}",95.0,ABZ,LGW
4,"{'ACE', 'AGP'}",130.0,ACE,AGP
...,...,...,...,...
1464,"{'ZRH', 'OTP'}",150.0,ZRH,OTP
1465,"{'ZRH', 'PRN'}",127.5,ZRH,PRN
1466,"{'ZRH', 'RVN'}",205.0,ZRH,RVN
1467,"{'ZRH', 'STR'}",45.0,ZRH,STR


## Import airport codes

In [53]:
# new dataset, which is the same but without the encoding error:
airports = pd.read_csv('./datasets/fixed-airports-codes.csv')
airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,K00A,NaN,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,00AN,NaN,00AN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78148,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
78149,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN
78150,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,87TX,NaN,87TX,NaN,NaN,NaN
78151,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,NaN,NaN,NaN,NaN,NaN,"ZZZW, ZZZW, ZYW, YK96"


In [54]:
# airports = pd.read_csv('./datasets/airport-codes.csv')
# airports

In [55]:
cities = airports[['iata_code', 'municipality']]
cities

,iata_code,municipality
0,NaN,Bensalem
1,NaN,Leoti
2,NaN,Anchor Point
3,NaN,Harvest
4,NaN,King Salmon
...,...,...
78148,NaN,Sealand
78149,NaN,Grande Glorieuse
78150,NaN,Blum
78151,NaN,(Old) Scandium City


In [56]:
# add departure city names
flights = pd.merge(flights, cities, left_on='departure airport', right_on='iata_code', how='left')
# remove the redundant column
flights.drop('iata_code', axis=1, inplace=True)
# rename the column
flights.rename(columns={'municipality': 'departure city'}, inplace=True)
# add arrival city names
flights = pd.merge(flights, cities, left_on='arrival airport', right_on='iata_code', how='left')
# remove the redundant column
flights.drop('iata_code', axis=1, inplace=True)
# rename the column
flights.rename(columns={'municipality': 'arrival city'}, inplace=True)
flights

,route,time,departure airport,arrival airport,departure city,arrival city
0,"{'AAL', 'AMS'}",85.0,AAL,AMS,Aalborg,Amsterdam
1,"{'AAQ', 'LED'}",170.0,AAQ,LED,Krasnyi Kurgan,St. Petersburg
2,"{'AMS', 'ABZ'}",85.0,AMS,ABZ,Amsterdam,Aberdeen
3,"{'ABZ', 'LGW'}",95.0,ABZ,LGW,Aberdeen,"Gatwick, Surrey"
4,"{'ACE', 'AGP'}",130.0,ACE,AGP,San Bartolomé,Málaga
...,...,...,...,...,...,...
1464,"{'ZRH', 'OTP'}",150.0,ZRH,OTP,Zurich,Otopeni
1465,"{'ZRH', 'PRN'}",127.5,ZRH,PRN,Zurich,Prishtina
1466,"{'ZRH', 'RVN'}",205.0,ZRH,RVN,Zurich,Rovaniemi
1467,"{'ZRH', 'STR'}",45.0,ZRH,STR,Zurich,Stuttgart


Does the 'municipality' variable refer to the actual administrative division that the airport is located in, or the big city it's next to? Let's check.

In [57]:
flights[flights['departure city'].str.contains('London')]

,route,time,departure airport,arrival airport,departure city,arrival city
698,"{'LHR', 'ABZ'}",95.0,LHR,ABZ,London,Aberdeen
699,"{'LHR', 'AGP'}",185.0,LHR,AGP,London,Málaga
700,"{'LHR', 'AMS'}",85.0,LHR,AMS,London,Amsterdam
701,"{'LHR', 'ARN'}",160.0,LHR,ARN,London,Stockholm
702,"{'LHR', 'ATH'}",245.0,LHR,ATH,London,Spata-Artemida
...,...,...,...,...,...,...
1201,"{'STN', 'VRN'}",122.5,STN,VRN,London,Caselle (VR)
1202,"{'STN', 'WMI'}",135.0,STN,WMI,London,Warsaw
1203,"{'STN', 'WRO'}",125.0,STN,WRO,London,Wrocław
1204,"{'STN', 'ZAG'}",135.0,STN,ZAG,London,Zagreb


### Save extended flight routes

In [58]:
flights.to_csv("./datasets/flight_times_extended.csv")

Save airport cities we have so far

In [59]:
airport_cites = set(flights["departure city"]) | set(flights["arrival city"])
airport_cites = pd.DataFrame(airport_cites)
airport_cites.to_csv("./datasets/airport_cities.csv",index=False, header=False)
airport_cites[0]

0                     Gibraltar
1      Montpellier/Méditerranée
2                       Kittilä
3                      Dortmund
4                      Jasionka
                 ...           
299                      Beslan
300                        Pula
301                 Southampton
302                     Bologna
303                  Luxembourg
Name: 0, Length: 304, dtype: object

# Analysis of Train and Airports names of cities

Now we need to check that the train and plane datasets are using the same city names. We'll do this by checking that the city names in the plane dataset are a subset of the city names in the train dataset.

Import of train routes preprocessed in "Rail transport performance.ipnyb"

In [19]:
airport_cities = pd.read_csv("../datasets/airport_cities.csv",header=None)
airport_cities.rename({0:"city"}, axis=1, inplace=True)
airport_cities

,city
0,Gibraltar
1,Montpellier/Méditerranée
2,Kittilä
3,Dortmund
4,Jasionka
...,...
299,Beslan
300,Pula
301,Southampton
302,Bologna


In [4]:
trains  = pd.read_csv('../datasets/train_routes.csv')
train_cities = pd.read_csv("../datasets/train_cities.csv")
trains = trains.iloc[:,1:]
train_cities

,city
0,St. Gallen
1,Cremona
2,Gütersloh
3,Basel
4,Salzburg
...,...
656,Bruxelles
657,Parla
658,Iserlohn
659,Harrogate


In [22]:
airport_cities_set  =set(airport_cities["city"])
airport_cities_set
train_cities_set  =set(train_cities["city"])
train_cities_set

{"'s-Hertogenbosch",
 'A Coruña',
 'Aachen',
 'Aberdeen',
 'Acerra',
 'Acireale',
 'Aix-en-Provence',
 'Albacete',
 'Alcalá de Henares',
 'Aldwick',
 'Alessandria',
 'Alicante/Alacant',
 'Alkmaar',
 'Almada',
 'Almelo',
 'Almere',
 'Alphen aan den Rijn',
 'Amersfoort',
 'Amiens',
 'Amsterdam',
 'Ancona',
 'Angers',
 'Annecy',
 'Antibes',
 'Antwerpen',
 'Anzio',
 'Apeldoorn',
 'Arad',
 'Arcozelo',
 'Arezzo',
 'Arnhem',
 'Arras',
 'Ashford',
 'Assen',
 'Asti',
 'Augsburg',
 'Avellino',
 'Avignon',
 'Aylesbury',
 'Ayr',
 'Bacău',
 'Bad Soden am Taunus',
 'Badajoz',
 'Bagheria',
 'Baia Mare',
 'Bamberg',
 'Bangor',
 'Banská Bystrica',
 'Barcelona',
 'Bari',
 'Barletta',
 'Barnsley',
 'Barreiro',
 'Basel',
 'Basildon',
 'Basingstoke and Deane',
 'Bath',
 'Battipaglia',
 'Bayonne',
 'Bayreuth',
 'Bedford',
 'Belfast',
 'Belfort',
 'Benrath',
 'Bergamo',
 'Bergen op Zoom',
 'Berlin',
 'Bern',
 'Besançon',
 'Beverwijk/Heemskerk',
 'Białystok',
 'Biel/Bienne',
 'Bielefeld',
 'Bielsko-Biała',
 '

Which cities are in the airport dataset but not the train dataset? 232

In [28]:
airport_train_cities = airport_cities_set & train_cities_set
airport_cities_only = airport_cities_set - train_cities_set
train_cities_only = train_cities_set - airport_cities_set

pd.DataFrame([{"Both":len(airport_train_cities),
              "Airport": len(airport_cities_only),
              "Train": len(train_cities_only)}])

,Both,Airport,Train
0,72,232,589


## Edition of train and airport cities

we create a mapping between old and new names in order to match cities from train and airports

In [42]:
flight_routes = pd.read_csv("../datasets/flight_times_extended.csv")
flight_routes = flight_routes.iloc[:,1:]
flight_routes.iloc[:,2:].head()

,departure airport,arrival airport,departure city,arrival city
0,AAL,AMS,Aalborg,Amsterdam
1,AAQ,LED,Krasnyi Kurgan,St. Petersburg
2,AMS,ABZ,Amsterdam,Aberdeen
3,ABZ,LGW,Aberdeen,"Gatwick, Surrey"
4,ACE,AGP,San Bartolomé,Málaga


In [78]:
def get_mapping_train(train_cities, cities_to_transform):
    """
     cities_to_transform - set of cities that don't match with airport cities
    """
    TRAIN_CITIES = train_cities.copy()
    TRAIN_CITIES["old_name"] = TRAIN_CITIES["city"]
    TRAIN_CITIES = TRAIN_CITIES.loc[ TRAIN_CITIES["old_name"].isin(cities_to_transform)]
    return TRAIN_CITIES

def get_mapping_airport(flight_routes, cities_to_transform):
    """
        flight_routes are used as we want to hold airport codes
    """
    airport1 = flight_routes[["departure airport","departure city"]].copy()
    airport2 = flight_routes[["arrival airport","arrival city"]].copy()

    airport1.rename({"departure airport":"code",
                                "departure city":"city"}, inplace=True, axis=1)
    airport2.rename({"arrival airport":"code",
                                    "arrival city":"city"}, inplace=True, axis=1)


    AIRPORTS = pd.concat([airport1, airport2], ignore_index=True )
    AIRPORTS.drop_duplicates(inplace=True)
    
    AIRPORTS = AIRPORTS.loc[AIRPORTS['city'].isin(cities_to_transform)]
    return AIRPORTS

def transform_airport_cities(airport):
    """
        airport_cities:
            - remove "()"
            - remove all after " "
    """
    pass
    

def transform_train_cities(df):
    pass

In [82]:
# train_cities["city"].isin(train_cities_only)

# subsetting data for reedition/mapping - TRAINS | AIRPORTS
TRAIN_CITIES = get_mapping_train(train_cities, train_cities_only)
# TRAIN_CITIES

AIRPORTS = get_mapping_airport(flight_routes, airport_cities_only)
AIRPORTS

,code,city
0,AAL,Aalborg
1,AAQ,Krasnyi Kurgan
4,ACE,San Bartolomé
20,ACI,Saint Anne
22,AER,Sochi
...,...,...
2543,PLQ,Palanga
2695,STW,Stavropol
2719,PIX,Pico Island
2837,KLU,Klagenfurt am Wörthersee


,city,old_name
0,St. Gallen,St. Gallen
1,Cremona,Cremona
2,Gütersloh,Gütersloh
3,Basel,Basel
5,Coimbra,Coimbra
...,...,...
655,Karlsruhe,Karlsruhe
656,Bruxelles,Bruxelles
657,Parla,Parla
658,Iserlohn,Iserlohn


Names in Airports, not in train_stations

Apply following changes:
- eliminate "()" parenthesis
- leave only names on the left side from the name

In [29]:
# matchin the cities
df_airport_only = pd.DataFrame(airport_cities_only)

{'Aalborg',
 'Aarhus',
 'Ajaccio/Napoléon Bonaparte',
 'Alexandroupolis',
 'Alghero',
 'Alicante',
 'Almería',
 'Andenes',
 'Archangelsk',
 'Astrakhan',
 'Balice',
 'Banja Luka',
 'Bastia/Poretta',
 'Belgrade',
 'Bergen',
 'Beslan',
 'Bilbao',
 'Billund',
 'Bodø',
 'Bolzano (BZ)',
 'Bordeaux/Mérignac',
 'Bragança',
 'Brønnøy',
 'Bugulma',
 'Burgas',
 'Béziers/Vias',
 'Cagliari',
 'Calvi/Sainte-Catherine',
 'Cascais',
 'Caselle (VR)',
 'Caselle Torinese (TO)',
 'Castellón de la Plana',
 'Castletown',
 'Cheboksary',
 'Chelyabinsk',
 'Cherepovets',
 'Chios Island',
 'Chita',
 'Chişinău',
 'Cluj-Napoca',
 'Copenhagen',
 'Corvera',
 'Craiova',
 'Crete Island',
 'Crotone',
 'Culleredo',
 'Debrecen',
 'Dortmund',
 'Dubrave Gornje',
 'El Matorral',
 'Evenes',
 'Faro',
 'Feldkirchen bei Graz',
 'Ferno (VA)',
 'Figari Sud-Corse',
 'Florø',
 'Frankfurt am Main (Lautzenhausen)',
 'Funchal',
 'Gatwick, Surrey',
 'Gelendzhik',
 'Geneva',
 'Genova (GE)',
 'Gibraltar',
 'Goleniow',
 'Gothenburg',
 'Gr

In [30]:
train_cities_only

{"'s-Hertogenbosch",
 'A Coruña',
 'Aachen',
 'Acerra',
 'Acireale',
 'Aix-en-Provence',
 'Albacete',
 'Alcalá de Henares',
 'Aldwick',
 'Alessandria',
 'Alicante/Alacant',
 'Alkmaar',
 'Almada',
 'Almelo',
 'Almere',
 'Alphen aan den Rijn',
 'Amersfoort',
 'Amiens',
 'Angers',
 'Annecy',
 'Antibes',
 'Antwerpen',
 'Anzio',
 'Apeldoorn',
 'Arad',
 'Arcozelo',
 'Arezzo',
 'Arnhem',
 'Arras',
 'Ashford',
 'Assen',
 'Asti',
 'Augsburg',
 'Avellino',
 'Avignon',
 'Aylesbury',
 'Ayr',
 'Bad Soden am Taunus',
 'Badajoz',
 'Bagheria',
 'Baia Mare',
 'Bamberg',
 'Bangor',
 'Banská Bystrica',
 'Barletta',
 'Barnsley',
 'Barreiro',
 'Basel',
 'Basildon',
 'Basingstoke and Deane',
 'Bath',
 'Battipaglia',
 'Bayonne',
 'Bayreuth',
 'Bedford',
 'Belfort',
 'Benrath',
 'Bergamo',
 'Bergen op Zoom',
 'Berlin',
 'Bern',
 'Besançon',
 'Beverwijk/Heemskerk',
 'Białystok',
 'Biel/Bienne',
 'Bielefeld',
 'Bielsko-Biała',
 'Bisceglie',
 'Blackburn with Darwen',
 'Blackpool',
 'Blanchardstown',
 'Blantyre',